In [7]:
# Install dependencies
import json
import numpy as np
import pandas as pd
import os
from math import inf

In [8]:
ip_list = []
rtt_matrix = {}
ip_hop_matrix = {}

In [9]:
for root, dirs, files in os.walk("PT"):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)

            with open(file_path, "r") as f:
                try:
                    for line in f:
                        try:
                            data = json.loads(line)
                            destination_ip_responded = data["destination_ip_responded"]

                            if not destination_ip_responded:
                                continue

                            src_ip = data["src_addr"]
                            dst_ip = data["dst_addr"]

                            if src_ip not in ip_list:
                                ip_list.append(src_ip)
                                rtt_matrix[src_ip] = {}
                                ip_hop_matrix[src_ip] = {}

                            if dst_ip not in ip_list:
                                ip_list.append(dst_ip)
                                rtt_matrix[dst_ip] = {}
                                ip_hop_matrix[dst_ip] = {}

                            last_hop = data["result"][-1]
                            num_ip_hop = last_hop["hop"]
                            rtt = min(hop.get("rtt", inf) for hop in last_hop["result"])

                            min_rtt = min(rtt, rtt_matrix[src_ip].get(dst_ip, inf))
                            min_ip_hop = min(num_ip_hop, ip_hop_matrix[src_ip].get(dst_ip, inf))

                            rtt_matrix[src_ip][dst_ip] = min_rtt
                            rtt_matrix[dst_ip][src_ip] = min_rtt

                            ip_hop_matrix[src_ip][dst_ip] = min_ip_hop    
                            ip_hop_matrix[dst_ip][src_ip] = min_ip_hop                    

                        except json.JSONDecodeError:
                            print(f"Skipping invalid JSON line in {file_path}")

                except Exception as e:
                    print(f"Error reading {file_path}: {e}")

In [10]:
rtt_df = pd.DataFrame(rtt_matrix).fillna("NA")
ip_hop_df = pd.DataFrame(ip_hop_matrix).fillna("NA")

In [11]:
rtt_df

,192.168.1.93,85.246.212.230,176.78.84.166,85.241.7.253,82.154.125.226,185.32.189.249,85.246.97.232,185.32.189.9,95.93.136.7,109.51.253.8,...,192.168.1.65,192.168.1.88,192.168.10.121,192.168.254.81,192.168.1.68,89.109.72.117,10.10.60.6,192.168.1.239,10.16.10.65,192.168.1.228
85.246.212.230,7.858,NA,NA,NA,NA,5.015,NA,5.21,12.218,NA,...,13.707,8.461,11.251,13.117,13.734,24.072,9.059,21.681,9.087,NA
176.78.84.166,13.799,NA,NA,NA,NA,10.644,NA,10.673,16.883,NA,...,17.546,13.145,15.099,17.329,17.838,32.65,13.745,20.579,12.953,NA
85.241.7.253,10.303,NA,NA,NA,NA,3.978,NA,5.126,9.789,NA,...,14.121,9.201,10.168,12.665,12.73,28.415,9.73,19.996,7.942,NA
82.154.125.226,11.941,NA,NA,NA,NA,5.055,NA,4.353,12.255,NA,...,15.375,10.578,11.669,13.212,13.776,25.161,10.775,20.824,8.726,NA
185.32.189.249,3.317,5.015,10.644,3.978,5.055,NA,1.766,0.287,7.708,5.625,...,12.289,5.94,8.392,9.723,10.577,21.839,7.408,12.853,5.204,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95.94.30.158,NA,NA,NA,NA,NA,9.868,9.47,10.048,17.032,10.68,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
89.155.57.136,NA,NA,NA,NA,NA,14.077,20.405,16.868,22.948,16.315,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
192.168.1.228,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
85.245.77.205,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [12]:
ip_hop_df

,192.168.1.93,85.246.212.230,176.78.84.166,85.241.7.253,82.154.125.226,185.32.189.249,85.246.97.232,185.32.189.9,95.93.136.7,109.51.253.8,...,192.168.1.65,192.168.1.88,192.168.10.121,192.168.254.81,192.168.1.68,89.109.72.117,10.10.60.6,192.168.1.239,10.16.10.65,192.168.1.228
85.246.212.230,8.0,NA,NA,NA,NA,8.0,NA,8.0,8.0,NA,...,8.0,7.0,8.0,8.0,8.0,10.0,9.0,13.0,9.0,NA
176.78.84.166,7.0,NA,NA,NA,NA,7.0,NA,7.0,7.0,NA,...,8.0,7.0,8.0,8.0,8.0,10.0,9.0,13.0,9.0,NA
85.241.7.253,8.0,NA,NA,NA,NA,8.0,NA,8.0,8.0,NA,...,8.0,7.0,8.0,8.0,8.0,10.0,9.0,13.0,9.0,NA
82.154.125.226,8.0,NA,NA,NA,NA,8.0,NA,8.0,8.0,NA,...,8.0,7.0,8.0,8.0,8.0,10.0,9.0,13.0,9.0,NA
185.32.189.249,6.0,8.0,7.0,8.0,8.0,NA,7.0,1.0,6.0,6.0,...,6.0,6.0,6.0,6.0,6.0,8.0,7.0,11.0,8.0,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95.94.30.158,NA,NA,NA,NA,NA,6.0,7.0,6.0,5.0,5.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
89.155.57.136,NA,NA,NA,NA,NA,6.0,8.0,6.0,5.0,5.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
192.168.1.228,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
85.245.77.205,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
